In [3]:
import shutil
from decimal import Decimal
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
import os

In [4]:
from nautilus_trader.backtest.node import (
    BacktestDataConfig,
    BacktestEngineConfig,
    BacktestNode,
    BacktestRunConfig,
    BacktestVenueConfig,
)

In [5]:
from nautilus_trader.test_kit.providers import CSVBarDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.model.instruments import FuturesContract
from nautilus_trader.model.data import BarType
from nautilus_trader.core.datetime import dt_to_unix_nanos

In [6]:
# this will be my default data directory from now on, see if you need to
# add it to env variables or something
DATA_DIR = "/home/Downloads/data"

SAMPLE_CSV = (
    "/home/mattia-carella/Downloads/data/csv/firstrate"
    "/indi_arch_fut_1d/E6_M24_1day.txt"
)
df = CSVBarDataLoader.load(
    SAMPLE_CSV,
    index_col=0,
    header=None,
    names=["timestamp", "open", "high", "low", "close", "volume", "oi"],
    # usecols=["open", "high", "low", "close", "volume"],
    parse_dates=True,
)[["open", "high", "low", "close", "volume"]]
df.tail()

,open,high,low,close,volume
timestamp,,,,,
2024-06-11,1.0768,1.07770,1.07230,1.07465,417297
2024-06-12,1.0743,1.08555,1.07375,1.08170,569877
2024-06-13,1.0814,1.08180,1.07345,1.07410,368141
2024-06-14,1.0738,1.07455,1.06680,1.06995,100280
2024-06-17,1.0704,1.07190,1.06870,1.07145,15995


## initialize intrument and bar wrangler


In [7]:
# equivalent to FuturesContract(...)
INSTRUMENT = TestInstrumentProvider.future(
    "E6M4", "E6"
)  # activation and expiration are wrong, price precision must be fixed

bar_type = BarType.from_str("E6M4.XCME-1-DAY-LAST-EXTERNAL")
wrangler = BarDataWrangler(bar_type, INSTRUMENT)
bars = wrangler.process(df)
# display the last bar
bars[-1]

Bar(E6M4.XCME-1-DAY-LAST-EXTERNAL,1.07,1.07,1.07,1.07,15995,1718582400000000000)

## creating a parquet data catalog


In [8]:
# Load environment variables
load_dotenv()

CATALOG_PATH = Path(os.getenv("NAUTILUS_CATALOG_PATH"))
CATALOG_PATH.mkdir(parents=True, exist_ok=True)

TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'NoneType'

In [ ]:
catalog = ParquetDataCatalog(CATALOG_PATH)

# adesso funziona
basename_template = f"{INSTRUMENT.symbol}"

# write instrument and bars to the catalog
catalog.write_data([INSTRUMENT])  # , basename_template=basename_template)
catalog.write_data(bars)  # , basename_template=basename_template)

In [ ]:
catalog.instruments()[0]

## Reading the catalog


In [ ]:
bars = catalog.bars()